## info gain: IG = E(parent) - [weighted average] * E(children)
## entropy : E = -sum(p(X) * log2(p(X)))
    
### p(X) = #x / n

In [67]:
import numpy as np
import pandas as pd
from sklearn import datasets
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


In [51]:
class Node:
    def __init__(self, feature = None ,threshold = None ,left = None , right = None , value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_node_leaf(self):
        return self.value is not None
        
class DecisionTree:
    def __init__(self, max_depth = 100 , min_sample = 2 , n_feature = None):       
        self.max_depth = max_depth
        self.min_sample = min_sample
        self.n_feature = n_feature
        self.root = None
        
    def fit(self, X,y):
        self.n_feature = X.shape[1] if not self.n_feature else min(X.shape[1] , self.n_feature)
        self.root = self.grow_tree(X,y)
        
    def grow_tree(self,X,y , depth = 0):
        n_sample , n_feat = X.shape
        n_labels = len(np.unique(y))
        
        # check the stopping criteria
        if depth >= self.max_depth or  n_labels == 1 or n_sample < self.min_sample:
            value_leaf = self.most_best_labels(y)
            return Node(value=value_leaf)
            
        #find best node
        
        idx_feats = np.random.choice(n_feat , self.n_feature , replace=False)
        
        best_feat , best_thread = self.most_best_feat(X,y,idx_feats)
            
        # create node child
        left_idxs, right_idxs = self._split(X[:,best_feat] , best_thread)
        
        left = self.grow_tree(X[left_idxs,:] , y[left_idxs] , depth+1)
        right = self.grow_tree(X[right_idxs,:] , y[right_idxs], depth+1)
        
        return Node(best_feat , best_thread , left , right)
        
        
    
    def most_best_feat(self,X,y,idx_feats):
        best_gain = -1
        
        split_feat , split_thr = None , None
        for idx_feat in idx_feats:
            X_column = X[:,idx_feat]
            thred = np.unique(X_column)
            
            for thr in thred:
                #caculator info gain
                gain = self.info_gain(X_column , y , thr)
                
                if gain > best_gain:
                    best_gain = gain
                    split_feat = idx_feat
                    split_thr = thr
                    
        return split_feat, split_thr
                
        
    def info_gain(self,X_column , y ,threshold):
        # caculator parent child
        parent_leaf = self.entropy(y)
        
        idx_left , idx_right = self._split(X_column , threshold)

        n_l , n_r = len(idx_left) , len(idx_right)
        
        if n_l == 0 or n_r == 0:
            return 0
        
        n = len(y)
        
        e_l , e_r = self.entropy(y[idx_left]) , self.entropy(y[idx_right])
        
        child_leaf = (n_l / n) * e_l + (n_r / n) * e_r
        
        infomation_gain = parent_leaf - child_leaf
        
        return infomation_gain
        
        
        
    def _split(self, X_column , threshold):
        left = np.argwhere(X_column <= threshold).flatten()
        right = np.argwhere(X_column > threshold).flatten()
        return left , right
        
    
        
        
    def entropy(self,y):
        hist = np.bincount(y)
        hp = hist / len(y)
        return - np.sum([p * np.log(p) for p in hp if p > 0])
            
    def most_best_labels(self,y):
        return np.argmax(np.bincount(y))
    
    def predict(self,X):
        
        return [self.find_node(x , self.root) for x in X]
    
    def find_node(self,x , node):
        if node.is_node_leaf():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self.find_node(x , node.left)
        return self.find_node(x , node.right)
        
    
        
        
        
    

In [61]:
from sklearn import datasets
from sklearn.model_selection import train_test_split


data = datasets.load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1
)


In [62]:
tree = DecisionTree(max_depth=10)
tree.fit(X_train, y_train)

In [63]:
pred = tree.predict(X_test)

In [64]:
def accuracy(y_pred , y):
    return np.sum(y_pred == y) / len(y)

In [65]:
accuracy(pred , y_test)

0.9385964912280702

In [69]:
dtree = DecisionTreeClassifier(max_depth= 10)
dtree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=10)

In [71]:
pred = dtree.predict(X_test)

In [72]:
accuracy(pred , y_test)

0.9473684210526315